In [3]:
pip install pinecone-client pinecone-text pinecone-notebooks langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s  0:00:02 eta 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [langchain_community]ngchain_community]
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain-huggingface pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 2.5 MB/s  0:00:0036m-:--:--
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pinecone]2/3 [pinecone]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
load_dotenv()
from langchain_huggingface import HuggingFaceEndpointEmbeddings, HuggingFaceEndpoint, ChatHuggingFace

/Volumes/New Volume/learning-langraph/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
index_name = "hybrid-search-langchain-pinecone"
#2 Initialize Pinecone
pinecone_api_key = os.getenv("PINE_CONE_API_KEY")
hugging_face_token = os.getenv("HUGGING_FACE_TOKEN")
pinecone = Pinecone(api_key=pinecone_api_key )

#3 Create Pinecone index
if not pinecone.has_index(index_name):
    pinecone.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct", # Using dotproduct for sparse values
        spec=ServerlessSpec(cloud="aws", region="us-east-1")

    )

In [3]:
index = pinecone.Index(name=index_name)
embaddings_model = HuggingFaceEndpointEmbeddings(
    repo_id="BAAI/bge-small-en",
    huggingfacehub_api_token=hugging_face_token,
)


/Volumes/New Volume/learning-langraph/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pinecone_text.sparse import BM25Encoder

sparse_encoder = BM25Encoder().default()

In [5]:
sentences = [
    "LangChain is a framework for developing applications powered by language models.",
    "Pinecone is a vector database that makes it easy to add semantic search capabilities to your applications.",
    "Hybrid search combines both dense and sparse retrieval methods to improve search accuracy and relevance."
]

#4 Upsert data into Pinecone index
# applying tf-idf encoding to the sentences
sparse_encoder.fit(sentences)

#store the sparse
sparse_encoder.dump("bm25_values.json")

#load the sparse values
sparse_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 284.47it/s]


In [6]:
retriever  = PineconeHybridSearchRetriever(
    embeddings=embaddings_model,
    sparse_encoder=sparse_encoder,
    index=index
)


In [9]:
retriever.add_texts([
    "LangChain is a framework for developing applications powered by language models.",
    "Pinecone is a vector database that makes it easy to add semantic search capabilities to your applications.",
    "Hybrid search combines both dense and sparse retrieval methods to improve search accuracy and relevance."
])

retriever.add_texts([
    "LangChain enables developers to build applications that can understand and generate human-like text.",
    "Pinecone provides a fully managed service for storing and querying vector embeddings at scale.",
    "By leveraging both dense embeddings and sparse representations, hybrid search can deliver more relevant results."
])

100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


In [10]:
retriever.invoke("What is LangChain?")

[Document(metadata={'score': 0.720260859}, page_content='LangChain is a framework for developing applications powered by language models.'),
 Document(metadata={'score': 0.687145412}, page_content='LangChain enables developers to build applications that can understand and generate human-like text.'),
 Document(metadata={'score': 0.390558243}, page_content='Hybrid search combines both dense and sparse retrieval methods to improve search accuracy and relevance.'),
 Document(metadata={'score': 0.386444092}, page_content='Pinecone provides a fully managed service for storing and querying vector embeddings at scale.')]